In [1]:
from tqdm import tqdm
from bs4 import BeautifulSoup 
import requests
from joblib import Parallel, delayed
import pickle
import pandas as pd
import numpy as np

C:\Users\canic\AppData\Local\Temp\ipykernel_35264\3030957463.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
category_link = 'https://starwars.fandom.com/wiki/Category:Canon_articles'  # all canon articles
original_link = 'https://starwars.fandom.com'

pages = {}
page_num = 1

while category_link is not None:
    req = requests.get(category_link)
    soup = BeautifulSoup(req.content, "html.parser")
    
    urls = soup.find_all('a', class_='category-page__member-link') # only get ones with class as category-page__member-link 
    links_before = len(pages)
    for link in urls:
        url = original_link + link.get('href')
        # We need to only get charachters and not categories, currently we are getting all links. We use the key to filter out categories:
        key = url.split('/')[-1]
        if 'Category:' not in key:
            pages[key] = url
        
    print(f'Page {page_num} - {len(pages) - links_before} next page: ({category_link})')
    page_num += 1

    # get next page button using the class
    next_urls = soup.find_all("a", class_='category-page__pagination-next') # In the code on the website we see the letter class for the next page button is category-page__pagination-next
    if next_urls:
        new_url = next_urls[0].get('href')
        if new_url == category_link:
            break 
        else:
            category_link = new_url
    else:
        break
        
print(f'Number of pages: {len(pages)}')



Page 1 - 199 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles)
Page 2 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=2-MAL+personal+comlink%0A2-MAL+personal+comlink)
Page 3 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=4-LOM%27s+DLT-19+heavy+blaster+rifle%0A4-LOM%27s+DLT-19+heavy+blaster+rifle)
Page 4 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=88-Z%0A88-Z)
Page 5 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=Absolution%0AAbsolution)
Page 6 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=Aduba-3%0AAduba-3)
Page 7 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=Airbrush%0AAirbrush)
Page 8 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_articles?from=Aleen+poster%0AAleen+poster)
Page 9 - 200 next page: (https://starwars.fandom.com/wiki/Category:Canon_ar

In [ ]:
# serialize the dictionary to a pickle file to make it easier to load later
with open("../data/pages_dict.pkl", "wb") as f:
    pickle.dump(pages, f)

In [13]:
# Check if random character is in the list
for i, x in pages.items():
    if x == "https://starwars.fandom.com/wiki/Boba_Fett":
        print("Boba Fett found")

Boba Fett found


---

In [ ]:
# deserialize the dictionary and print it out
with open("../data/pages_dict.pkl", "rb") as f:
    pages = pickle.load(f)
len(pages)

48789

In [15]:
# req = requests.get("https://starwars.fandom.com/wiki/Boba_Fett")
# soup = BeautifulSoup(req.content, "html.parser")

# # get the name of the character
# name = soup.find('h1',class_='page-header__title').text.strip()

# # Get sidebar with info
# sidebar = soup.find('aside')
# # collect all relevant info in the sidebar
# info = {"Name": name,
#         "homeworld": None,
#         "birth": None,
#         "species": None,
#         "gender": None,
#         "pronouns": None,
#         "donor": None,
#         "height": None,
#         "mass": None,
#         "hair": None,
#         "eyes": None,
#         "skin": None,
#         "families": None,
#         "parents": None,
#         "siblings": None,
#         "affiliations": None,
#         "masters": None,
#         "apprentices": None}

# for item in sidebar.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
#     #get the data:
#     info[item["data-source"]] = item.find('div', class_='pi-data-value pi-font').text.strip()

# # remove the references in format [X] from the data
# for key, value in info.items():
#     if value is not None:
#         info[key] = value.split('[')[0].strip()
#     else:
#         info[key] = None
# print(info)

In [16]:
# def process_page(url):
#     req = requests.get(url)
#     soup = BeautifulSoup(req.content, "html.parser")

#     # get the name of the character
#     name = soup.find('h1',class_='page-header__title').text.strip()

#     # Get sidebar with info
#     sidebar = soup.find('aside')
#     # collect all relevant info in the sidebar
#     info = {"homeworld": None,
#             "birth": None,
#             "species": None,
#             "gender": None,
#             "pronouns": None,
#             "donor": None,
#             "height": None,
#             "mass": None,
#             "hair": None,
#             "eyes": None,
#             "skin": None,
#             "families": None,
#             "parents": None,
#             "siblings": None,
#             "affiliations": None,
#             "masters": None,
#             "apprentices": None}

#     for item in sidebar.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
#         #get the data:
#         info[item["data-source"]] = item.find('div', class_='pi-data-value pi-font').text.strip()

#     # remove the references in format [X] from the data
#     for key, value in info.items():
#         if value is not None:
#             info[key] = value.split('[')[0].strip()
#         else:
#             info[key] = None
#     return name,info


# # Process pages in parallel
# characters = {}
# results = Parallel(n_jobs=-1)(
#     delayed(process_page)(pages[key]) for key in tqdm(pages.keys(), desc="Processing pages", unit="page", ncols=100)
# )

# # Collect results
# for name, info in results:
#     if name and info:
#         characters[name] = info

In [17]:
# # serialize the dictionary to a pickle file to make it easier to load later
# with open("charachters_info.pkl", "wb") as f:
#     pickle.dump(characters, f)

In [18]:
def process_page(url):
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")

    # get the name of the character
    name = soup.find('h1',class_='page-header__title').text.strip()

    # Get sidebar with info
    sidebar = soup.find('aside')
    # collect all relevant info in the sidebar
    info = {}

    for item in sidebar.find_all('div', class_='pi-item pi-data pi-item-spacing pi-border-color'):
        #get the data:
        info[item["data-source"]] = item.find('div', class_='pi-data-value pi-font').text.strip()

    # remove the references in format [X] from the data
    for key, value in info.items():
        if value is not None:
            info[key] = value.split('[')[0].strip()
        else:
            info[key] = None
    return name,info


# Process pages in parallel
characters = {}
results = Parallel(n_jobs=-1)(
    delayed(process_page)(pages[key]) for key in tqdm(pages.keys(), desc="Processing pages", unit="page", ncols=100)
)

# Collect results
for name, info in results:
    if name and info:
        characters[name] = info

Processing pages: 100%|█████████████████████████████████████| 48789/48789 [17:47<00:00, 45.72page/s]


In [ ]:
# serialize the dictionary to a pickle file to make it easier to load later
with open("../data/charachters_info_only_filled_att.pkl", "wb") as f:
    pickle.dump(characters, f)

---

In [ ]:
# deserialize the dictionary and print it out
with open("../data/charachters_info_only_filled_att.pkl", "rb") as f:
    characters = pickle.load(f)
len(characters)

42217

In [29]:
# create a pandas dataframe from the dictionary
df = pd.DataFrame.from_dict(characters, orient='index')
df

,manufacturer,type,cost,purpose,species,affiliation,gender,pronouns,hair,skin,...,albums,form,teams,depth,candidates,electorate,stars,arms,satellites,incubationperiod
"""Accu-Strike"" integrated targeting computer",Industrial Automaton,Targeting computer,"3,000 credits",Providing targeting data and improving accuracy,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Autoguard"" Cybernetic Reflex Suite",Czerka Arms,Droid brain defense system,"5,000 credits",Warn its user from incoming threats,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Black Hole Box"" T-11 Armored Databank",Astralor Corp.,Armored computer core,"1,100 credits",Protect information from a starship's full des...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Blinder"" 4B2 jamming array",Irilliad,Sensor jammer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"""Breaker"" heavy hydrospanner",Regallis Engineering,Hydrospanner,250 credits,Fastening bolts,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zabrak scalp necrosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Owauwaq Freight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Smoothie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Pipada Symphony Chamber Chorus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# giving first column a name so it is not empty
df.index.name = 'name'
df.reset_index(inplace=True)
df

,name,manufacturer,type,cost,purpose,species,affiliation,gender,pronouns,hair,...,albums,form,teams,depth,candidates,electorate,stars,arms,satellites,incubationperiod
0,"""Accu-Strike"" integrated targeting computer",Industrial Automaton,Targeting computer,"3,000 credits",Providing targeting data and improving accuracy,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"""Autoguard"" Cybernetic Reflex Suite",Czerka Arms,Droid brain defense system,"5,000 credits",Warn its user from incoming threats,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"""Black Hole Box"" T-11 Armored Databank",Astralor Corp.,Armored computer core,"1,100 credits",Protect information from a starship's full des...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"""Blinder"" 4B2 jamming array",Irilliad,Sensor jammer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Breaker"" heavy hydrospanner",Regallis Engineering,Hydrospanner,250 credits,Fastening bolts,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42212,Zabrak scalp necrosis,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42213,Owauwaq Freight,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42214,Smoothie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42215,Pipada Symphony Chamber Chorus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


As we see there are a lot of different attributes, to be exact 382 and we will therefore filter alot of them out as they are redundant!

In [31]:
# Count non-null values in each column
all_columns_count = df.count()
print(all_columns_count)

# Filter columns with more than 3000 non-null values
filtered_columns = all_columns_count[all_columns_count > 3000].index

# removing filled_coloumns column
filtered_columns = filtered_columns[filtered_columns != "filled_columns"]

df_filtered = df[filtered_columns]

# removing rows with all null values not including the name column
df_filtered = df_filtered.dropna(how='all', subset=[col for col in df_filtered.columns if col != 'name'])

# removing rows with less than 2 non-null values not including the name column
df_filtered = df_filtered.dropna(thresh=2, subset=[col for col in df_filtered.columns if col != 'name'])

df_filtered

name                42217
manufacturer         2447
type                10373
cost                  923
purpose              1357
                    ...  
electorate              1
stars                   1
arms                    1
satellites              1
incubationperiod        1
Length: 382, dtype: int64


,name,type,species,affiliation,gender,pronouns,hair,skin,homeworld,death,location,eyes,class
5,"""Changeling"" Mark 71NB",Racing starfighter,NaN,Ace Squadron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TIE/ba Baron Space Superiority Interceptor,Starfighter,NaN,First Order,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"""Impact"" repulsor cannon",Repulsor cannon,NaN,Law enforcement agencies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,"""Peacekeeper"" stun baton",Stun baton,NaN,Galactic Empire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,"""Renegade"" heavy blaster pistol",Heavy blaster pistol,NaN,Alliance to Restore the Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29198,Unidentified maintenance droid 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c. 232 BBY, Steady Wing, Haileap",NaN,NaN,Maintenance droid
29204,Unidentified protocol droid (Suli's cantina),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"29 ABY, Chaaktil",NaN,NaN,Protocol droid
29205,Unidentified RA-7 protocol droid (Nar Shaddaa),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 ABY,NaN,NaN,Protocol droid
29207,Unidentified service droid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,During the skirmish aboard the Sorca Retreat,NaN,NaN,Service droid


In [32]:
# Count non-null values in each column
all_columns_count = df_filtered.count()
print(all_columns_count)

name           20532
type            6185
species         9244
affiliation    16818
gender          7353
pronouns        5010
hair            3545
skin            5368
homeworld       2992
death           3074
location        3320
eyes            4262
class           1461
dtype: int64


In [33]:
# remove duplicates if any
df_filtered = df_filtered.drop_duplicates()
df_filtered

,name,type,species,affiliation,gender,pronouns,hair,skin,homeworld,death,location,eyes,class
5,"""Changeling"" Mark 71NB",Racing starfighter,NaN,Ace Squadron,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,TIE/ba Baron Space Superiority Interceptor,Starfighter,NaN,First Order,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,"""Impact"" repulsor cannon",Repulsor cannon,NaN,Law enforcement agencies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,"""Peacekeeper"" stun baton",Stun baton,NaN,Galactic Empire,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,"""Renegade"" heavy blaster pistol",Heavy blaster pistol,NaN,Alliance to Restore the Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29198,Unidentified maintenance droid 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"c. 232 BBY, Steady Wing, Haileap",NaN,NaN,Maintenance droid
29204,Unidentified protocol droid (Suli's cantina),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"29 ABY, Chaaktil",NaN,NaN,Protocol droid
29205,Unidentified RA-7 protocol droid (Nar Shaddaa),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0 ABY,NaN,NaN,Protocol droid
29207,Unidentified service droid,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,During the skirmish aboard the Sorca Retreat,NaN,NaN,Service droid


In [ ]:
df_filtered.to_csv('../data/filtered_charachter_info.csv', index=False)